# Dive into lambda calculus

David Beazley pycon 2019
https://www.youtube.com/watch?v=pkCLMl0e_0k

What's a function?

ELI5: 
f(x) = 2 * x

Replace all xs on the right with the given x on the left:

f(2) = 2 * 2 = 4


# Starting point: 
ONLY functions with single arguments... f(x) is ALL we have. NO control flow, numbers, nothing.

we can do things like:

def f(x): return x 

def f(x): return x(x) # since x is a function

and...:

In [1]:
def f(x):
    def g(y):
        return x(y)
    return g

In [2]:
f(print)("hi") # This is currying, more on that later... 

hi


In [3]:
# The above works because f(print) returns a function that will call print:
f(print)

<function __main__.f.<locals>.g(y)>

In [4]:
# Note the .g(y) there? we now have a function that will call print on the next argument passed in:
_('hi')

hi


That's an example for later, and not technically allowed in our system since we passed in a string (no strings, only single argument functions). We'll use some 'nice' things like strings just to see how things are working.

# What can we actually model with just single argument functions?

How about a switch? 

In [5]:
def left(a):
    def f(b):
        return a
    return f

def right(a):
    def f(b):
        return b
    return f


Given to things, return the left or return the right (similar to what we did above).
another way to think about it: return the argument passed to the initial function, or the argument passed into the function returned by the initial function. 

In [6]:
left('l')('r')

'l'

In [7]:
right('l')('r')

'r'

NOTE: We're not actually representing anything. It's simply a behavior. There's no bits or data structure beneath this

# Can we represent truth?
We could base it off our left and right examples. 
Truth is when we look at the first value? False is when we look at the second value? It's sort of arbitrary.
True is one option and false is the other.

In [8]:
def TRUE(x): return lambda y: x
def FALSE(x): return lambda y: y

'True' isn't a thing, nothing is concrete here. Instead of 0 and 1 it's just the first thing or the second thing...

## How about boolean logic...?
What if we wanted operators like not, and, or etc....

In [9]:
def NOT(x): return x(FALSE)(TRUE)

In [11]:
NOT(TRUE) # true returns the first thing, so:

<function __main__.FALSE(x)>

In [13]:
NOT(FALSE) # false returns the second thing...:

<function __main__.TRUE(x)>

In [14]:
def AND(x): 
    return lambda y: x(y)(x)

In [15]:
AND(TRUE)(FALSE)

<function __main__.FALSE(x)>

In [16]:
def OR(x):
    return lambda y: x(x)(y)

In [17]:
OR(TRUE)(FALSE)

<function __main__.TRUE(x)>

In [18]:
OR(FALSE)(TRUE)

<function __main__.TRUE(x)>

## Boolean logic with ONLY single argument function calls.
This is all coming from the behavior of the functions. We decided true was simple the first thing and false was the second thing.

# Now lets look at numbers!
Finger counting is how we all start. Can we use that same concept? A number is just how many times we call a function?

In [19]:
ZERO = lambda f: lambda x: x
ONE = lambda f: lambda x: f(x)
TWO = lambda f: lambda x: f(f(x))
THREE = lambda f: lambda x: f(f(f(x)))
FOUR = lambda f: lambda x: f(f(f(f(x))))

In [20]:
# A helper function so we can see actual numbers coming out:
def incr(x):
        return x + 1

Numbers aren't things (bits, whatever), they're *behaviors*. 

In [21]:
THREE(incr)(0) # Three calls of incrementing 0 by 1 --> 3

3

These are called Church numerals.

How do we handle zero...? Well if everything is based on how many times you call a function on an input, zero should just return the input after not calling a function at all:

In [22]:
ZERO = lambda f: lambda x: x

In [23]:
ZERO(incr)(0)

0

## Can we build math?

https://en.wikipedia.org/wiki/Peano_axioms

Our challenge is to build some notion of 'successor'. 
SUCC: takes a number --> number + 1

The API for our numbers is that they're some function f, called repeatedly. So the successor of a THREE, f(f(f(X))) would just be one more f() --> f(f(f(f(x)))) which is FOUR. 

In [24]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [25]:
SUCC(TWO)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [26]:
_(incr)(0)

3

Ok, we can a basic representation of numbers and a successor function. What about add?

Successor gives us the next number, so if we had x + y, we could get the sum by calling SUCC(x) y times.

In [27]:
ADD = lambda x: lambda y: x(SUCC)(y) 

In [28]:
ADD(ONE)(TWO)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [29]:
_(incr)(0)

3

What about multiplication? x * y --> x repetitions of f y times

In [30]:
MULT = lambda x: lambda y: lambda f: y(x(f))

In [31]:
MULT(TWO)(TWO)

<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>(f)>

In [32]:
_(incr)(0)

4

## A quick aside on the symbols:
Here's a breakdown of our approach to function definition. We started with the Python 'def' style and eventually got to the lambda style with no def.

In [33]:
def AND(x):
    def f(y):
        return x(y)(x)
    return f

# But also:
def AND(x):
    return lambda y: x(y)(x)

# But even just...:
AND = lambda x: lambda y: x(y)(x)

We can take this even further...

in classic lambda calculus we get something more like this:
AND = lambda x: lambda y: x(y)(x)
AND = lambda xy: x(y)(x)
AND = lambda xy: xyx
AND = lambda xy.xyx --- this what we'd see in any true lambda calculus paper etc...


There are some rules for naming/syntax of 'true' lambda calculus formatting:

Rule 1: you can rename variables only if the new name doesn't introduce a clash. 
known: "alpha conversion"

Rule 2: you substitute arguments:
(lambda xy.xyx)(ab)
--> lambda y.(ab)y(ab)
We only had one argument, (ab), which was directly substituted for x in the original function (xyx).
caveat: We cannot introduce name clashes! so arguments can't be the same as the args in the function def. We get around that by using rule 1.
known: "beta reduction"

Rule 3: You can make a function. 

David Hilbert, defining a formal symbol definition of mathematics. 'Can you derive mathematical truths from symbol manipulations'. 

Can all of math be dereived from formal systems?

Many things came out of this inluding: Goedel incompleteness theorum
Alonzo Church: lambda calculus, how do you mathematically describe algorithms?
Turing: lamnbda calc and turing machines are equivalent to eachother.

Can you prove all of math through algorithms? you cannot...

~20 years later, John McCarthy and LISP.


## Is there any way to build a data structure..?
How should we encode data in this lambda calculus world?

in LISP we have:
(cons 2 3) --> (2,3)
(car p)    --> 2
(cdr p)    --> 3

Can we build a data structure entirely out of a function definition?

Let's try a cons like funciton in normal Python:

In [34]:
def cons(a, b):
    def select(m):
        if m == 0:
            return a
        if m == 1:
            return b
    return select

In [35]:
p = cons(2,3)

In [36]:
p

<function __main__.cons.<locals>.select(m)>

In [37]:
p(0)

2

In [38]:
p(1)

3

p is a function that's holding the passed in values. 

This select statement is just like the left/right functions in the lambda calculus that we started out with. Let's try it out in lambda syntax.

In [39]:
CONS = lambda a: lambda b: (lambda s: s(a)(b))

In [40]:
p = CONS(1)(2)

In [41]:
p(left)

1

In [42]:
p(right)

2

The above would also work with TRUE/FALSE as they act the same way. Now we can define a CAR and CDR in the same way:

In [43]:
CAR = lambda p: p(left)
CDR = lambda p: p(right)

In [44]:
p


<function __main__.<lambda>.<locals>.<lambda>.<locals>.<lambda>(s)>

In [45]:
CAR(p)

1

In [46]:
CDR(p)

2

No data is being stored, this is purely a function..! With this we can built linked list type strutures. We can also try and tackle the problem of getting to the previous number.

What if to get to the previous number, we just built up these CONS pairs from (0, 0) all the way up to (n, n-1).


In [47]:
T = lambda p: CONS(SUCC(CAR(p)))(CAR(p))

In [48]:
a = FOUR(T)(CONS(ZERO)(ZERO))

In [49]:
CAR(a)(incr)(0)

4

In [50]:
CDR(a)(incr)(0)

3

Now we can define a PRED function that returns the predecessor of a number.

In [51]:
PRED = lambda n: CDR(n(T)(CONS(ZERO)(ZERO)))

In [52]:
a = FOUR(THREE)

In [53]:
a(incr)(0)

81

In [54]:
PRED(a)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [55]:
_(incr)(0)

80

Wildly impracticle. We'd never want to find a previous number by counting all the way back up to n by pairs (n, n-1)

Now in the same way that we built ADD with the SUCC function, we can build out SUB by applying the PRED function.

In [56]:
SUB = lambda x: lambda y: y(PRED)(x)

In [57]:
SUB(FOUR)(TWO)

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [58]:
_(incr)(0)

2

Lets build up some control flow type mechanisms by starting with testing for a number being equal to ZERO.

ZERO, by our definition of numbers, doesn't call the function f on some input x. Numbers always have some notion of an input function and an argument to apply that function to. ZERO doesn't apply it's function and just returns the argument.

ZERO = lambda f: lambda x: x --> just return x


In [59]:
ISZERO = lambda n: n(lambda f: FALSE)(TRUE) #

In [60]:
ISZERO(ZERO)

<function __main__.TRUE(x)>

In [61]:
ISZERO(FOUR)

<function __main__.FALSE(x)>

ZERO is a special case where the function is never called, and in this case the function just returns FALSE, indicating that the number is not ZERO. 

## At this point we've built out a mini "assembly code" of functions:

we have logic:
+ AND
+ OR
+ NOT

numbers/math:
+ SUCC
+ PRED
+ ADD
+ MUL
+ SUB

'data structures':
+ CONS
+ CAR
+ CDR

testing:
+ ISZERO

Now let's write more complicated things. How about factorial?

In [62]:
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n-1)

In [63]:
fact(4)

24

What about with our 'assembly language'? We have all the components necessary: PRED, ISZERO, MUL....

Note: ISZERO returns TRUE or FALSE, and in our environment, the behavior of truth is to return the first thing (FALSE the second thing).

In [64]:
FACT = lambda n: ISZERO(n)(ONE)(MULT(n)(FACT(PRED(n))))

In [65]:
FACT(THREE)

RecursionError: maximum recursion depth exceeded

Python does the arguments first, then proceeds with the path. Eager evaluation of arguments. This is causing both branches of our ISZERO to be evaluated, which leads to infinite recursion. 

Turn arguments into functions (lambda) then we only need to evaluate them if we go down that branch. This is just a hack to get around Python's eager evaluation...

In [66]:
LAZY_TRUE = lambda x: lambda y: x()
LAZY_FALSE = lambda x: lambda y: y()
ISZERO = lambda n: n(lambda f: LAZY_FALSE)(LAZY_TRUE)

In [67]:
FACT = lambda n: ISZERO(n)\
                        (lambda: ONE)\
                        (lambda: MULT(n)(FACT(PRED(n))))

In [68]:
a = FACT(THREE)

In [69]:
a(incr)(0)

6

When things evaluate/happen is a big focus in functional languages. Python doesn't really support this laziness of evaluaiton so we needed to hack it together by wrapping branches in functions. 

Lambda calculus doesn't support any global variables (because that requires variables). We've been cheating that a little bit by assigning lambdas to things like FACT and MULT etc...

This presents a big problem when doing recursion, since the name can't be globally known. How do we do recursion?

In [70]:
# Example:
fact = lambda n: 1 if n == 0 else n * fact(n-1)

In [71]:
fact(4)

24

Now how do we rewrite that without the self reference...? Well we can do what we know is bad and can just copy paste the code again...

In [72]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n-1))(fact)

We've made it accept an argument, but we can't use the fact name here.... So what to do... Well we do what we've been told never to do, we repeat the code....

In [73]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(n-1))\
        (lambda f: lambda n: 1 if n == 0 else n * f(n-1))

In [74]:
fact(3)

TypeError: unsupported operand type(s) for *: 'int' and 'function'

we've messed up the 'API' for this function. fact takes a function f first and a number n second. We're not doing that in the 'recursive' part of this. f is a two arguments function, so we need to pass it in... Oh boy is this ugly...

In [75]:
fact = (lambda f: lambda n: 1 if n == 0 else n * f(f)(n-1))\
        (lambda f: lambda n: 1 if n == 0 else n * f(f)(n-1))
# Now we've conformed to the 'API' that f takes 'two' arguments, a function and a number....

In [76]:
fact(4)

24

Now we have a recursive factorial function without using the 'fact' name above. We could even do this:

In [77]:
print( (lambda f: lambda n: 1 if n == 0 else n * f(f)(n-1))\
        (lambda f: lambda n: 1 if n == 0 else n * f(f)(n-1))(5))

120


## Y Combinator, the alternate way of doing this....

fixed point: function with some input, we get the same thing back.

example: sqrt(1) = 1

How are 'fixed points" related to recursion? Well it seems like they define the stoping point... Let's look back at our original factorial function and the first changes we made:

In [78]:
fact = lambda n: 1 if n==0 else n * fact(n-1)

In [79]:
# Then we did a trick with the variable name:
fact = (lambda f: lambda n: 1 if n==0 else n*f(n-1))(fact)

In [80]:
fact(5)

120

fact is on both sides of the equation. Let's take out the middle part and see what it looks like...:

In [81]:
R = (lambda f: lambda n: 1 if n==0 else n*f(n-1))

In [82]:
# now we have:
fact = R(fact)

Now this looks like fact is a fixed point of R. we pass something in, and get the same thing back. 1=sqrt(1). R of fact returns to us fact. This is a mathematical relationship, not an assignment or anything like that.

Now it's time for a leap!

Suppose there's a function Y that computes the fixed point of R (whatever it happens to be).
If such a thing existed, you could say:
Y(R) = R(Y(R))
This is just replacing 'fact' above. If we do our repeating ourselves recursive trick we'll end up with....


In [83]:
Y = lambda f: (lambda x: f(x(x)))(lambda x: f(x(x)))

In [84]:
fact = Y(R)

RecursionError: maximum recursion depth exceeded

Again, Python is causing some issues with this, so we need to make a quick hack. The eager argument evaluation problem. How do we fix it? Well we just wrap it in a function (decorators!)

We can even use this 'decorator' idea in the lambda calculus:

In [85]:
Y = lambda f: (lambda x: f(lambda z: x(x)(z)))(lambda x: f(lambda z: x(x)(z)))

In [86]:
fact = Y(R)

In [87]:
fact(5)

120

## Making use of Python numbers: church() and unchurch()
I'll add in some quick helper functions that will convert back and forth between Church numerals (how we're implmenting numbers here) and python integers.

In [108]:
# Recurse from n down to 0 adding a call to SUCC at each recursive call. 
def church(n):
    if n == 0:
        return ZERO
    else:
        return SUCC(church(n-1))

In [115]:
# Since church numerals represent repeated calls of a function, to get back an int from a church numeral we just
# pass it a plus_one function starting at 0.
def unchurch(n):
        plus_one = lambda x: x + 1
        return n (plus_one)(0)
        

In [112]:
nine = church(9)

In [113]:
nine

<function __main__.<lambda>.<locals>.<lambda>(f)>

In [114]:
unchurch(nine)

9